In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

In [16]:
data = pd.read_csv('checkins.dat', sep='|', header=None)
# data['id'] = data['id'].astype('int') # .loc[0, :].drop(axis=1) #['latitude'].map(lambda x: np.nan if x == ' ' else x)
for column in range(4):
    data[column] = pd.to_numeric(data[column], errors='coerce')

data=data.dropna().drop([0, 1, 2, 5], axis=1)

In [17]:
data.count()

3    396634
4    396634
dtype: int64

In [19]:
meanshift = MeanShift(n_jobs=10, bandwidth=0.1)
clust = meanshift.fit_predict(data.head(100000))

In [81]:
numbers = pd.Series(clust).value_counts()
idx = numbers[numbers > 15].index
centers = pd.DataFrame(meanshift.cluster_centers_)
centers = centers.loc[idx, :].copy()
centers.columns = ['latitude', 'longitude']

In [82]:
top_centers = np.array([
[33.751277, -118.188740], # (Los Angeles)
[25.867736, -80.324116], # (Miami)
[51.503016, -0.075479], # (London)
[52.378894, 4.885084], # (Amsterdam)
[39.366487, 117.036146], # (Beijing)
[-33.868457, 151.205134] # (Sydney)
])

In [83]:
def dist(x, y):
    return (x[0] - y[0])**2 + ((x[1]) - y[1])**2

for i in range(top_centers.shape[0]):
    centers[i] = 0.

for i in centers.index:
    for j in range(top_centers.shape[0]):
        centers.at[i, j] = dist(centers.loc[i, ['latitude', 'longitude']].values, top_centers[j, :])
        
centers['dist'] = 0.
for i in centers.index:
    centers.at[i, 'dist'] = np.min(centers.loc[i, range(top_centers.shape[0])])

In [84]:
ans = centers.sort_values(by='dist').head(1)[['latitude', 'longitude']].values
print ans

[[-33.86063043 151.20477593]]
